In [1]:
import fsps
import matplotlib.pyplot as plt
import numpy as np
from cloudyfsps import outObj as ob
from cloudyfsps.outObj import getColors, nColors
from cloudyfsps.generalTools import calcQ
import matplotlib as mpl
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
mpl.rc('font', size=16, family='serif', serif=[r'cmr10'], style='normal', variant='normal', stretch='normal', weight='heavy')
mpl.rc('legend', labelspacing=0.1, handlelength=2, fontsize=16)
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline

In [ ]:
def masspile(tage=0.001, z=0.0, cname='inferno', alpha=0.75, plt_cum=False, do_title=None,
             colors=None, xlabel=None, ylabel=None,
             masscut=120, masslims=[1., 2.0, 15.0, 120.0], ax=None, fontsize=16, **kwargs):
    sps.params['masscut'] = masscut
    sps.params['logzsol'] = z
    # Try to set any extra params
    for k, v in kwargs.items():
        try:
            sps.params[k] = v
        except(KeyError):
            pass
    # Make a pretty string with the FSPS parameters
    if do_title is None:
        titlestr = r'{0:.0f} Myr, log Z/Z$_{{\odot}}$ = {1:.1f}'.format(tage*1.e9/1.e6, z)
    else:
        titlestr = do_title
    # colors
    if colors is None:
        colors = sns.color_palette(cname, len(masslims))
    # get the total spectrum due to *all* stars
    wave, spec_tot = sps.get_spectrum(tage=tage, peraa=True)

    # Set up stellar mass ranges and arrays to hold output spectra
    mspec = np.zeros([len(sps.wavelengths), len(masslims)])
    dmspec = np.zeros_like(mspec)
    # set up figure and axis objects
    if ax is None:
        nfig, nax = plt.subplots(figsize=(8, 6))
    else:
        nax = ax
        nfig = plt.gcf()
    xlim = kwargs.get('xlim', (0.1e3, 2e5))
    nax.set_xlim(xlim)
    for i, mc in enumerate(masslims):
        # set the upper stellar mass limit
        sps.params['masscut'] = mc
        # get the spectrum at age = tage and store it
        w, spec = sps.get_spectrum(tage=tage, peraa=True)
        mspec[:, i] = spec.copy()
        if i == 0:
            continue
        dmspec[:, i] = spec - mspec[:, i-1]
    # this makes the plot limits work out for the fill between
    nax.loglog(w, spec_tot, lw=0.8, label='__None__', color='black', alpha=0.0)
    #for ind, color in zip(range(1, len(masslims))[::-1], colors):
    for ind, color in zip(range(0, len(masslims))[::-1], colors):
        if ind == 0:
            lab = r'$\mathrm{{M}}<{0:.0f}\mathrm{{M}}_{{\odot}}$'.format(masslims[ind])
            nax.fill_between(w, 0.0, mspec[:,ind], color=color, label=lab)
            continue
        else:
            lab = r'${0:.0f} < \mathrm{{M}} < {1:.0f}$'.format(masslims[ind-1], masslims[ind])
            #nax.fill_between(w, mspec[:,ind-1], mspec[:,ind-1]+dmspec[:,ind], color=color, label=lab)
            #both work, this is just cleaner
            nax.fill_between(w, mspec[:,ind-1], mspec[:,ind], color=color, label=lab)
    pl_legend = kwargs.get('pl_legend', True)
    if pl_legend:
        frameon=kwargs.get('frameon', False)
        markerfirst=kwargs.get('markerfirst', False)
        naxloc = kwargs.get('naxloc', 2)
        nax.legend(loc=naxloc, fontsize=fontsize, frameon=frameon, markerfirst=markerfirst)
    if xlabel is None:
        xlabel=r'Wavelength [$\,\mathrm{\AA}\,$]'
    if ylabel is None:
        ylabel=r'$\mathcal{F}_{\lambda}\ (\,\mathrm{L}_{\odot}/\mathrm{\AA}\,)$'
    nax.set_xlabel(xlabel, size=16)
    nax.set_ylabel(ylabel, size=16)
    ylim = kwargs.get('ylim', (1e-17, 1e-12))
    nax.set_ylim(ylim)
    nax.set_title(titlestr)
    return nax, mspec, dmspec

In [ ]:
sps = fsps.StellarPopulation(zcontinuous=1)
sps.params['imf_type'] = 2   # Kroupa IMF
sps.params['sfh'] = 0  # burst

# Figure 1

In [ ]:
z=0.0
tages = [1.e-3, 2.e-3, 3.e-3, 5.e-3, 10.e-3, 20.e-3]
masslims = [5., 10., 25., 35., 75., 120.]
colors = nColors(len(masslims), maxv=0.8)
fig, axes = plt.subplots(len(tages)/2, 2, figsize=(8, 5.), sharex=True, sharey=True)

for i, (tage, ax) in enumerate(zip(tages, fig.axes)):
    pl_leg=False
    xlabel=' '
    ylabel=' '
    if i == 1:
        ax.text(950., 1.e-4, r'I$_{\mathrm{H}}$', fontsize=16)
        ax.text(550., 1.e-4, r'I$_{\mathrm{He}}$', fontsize=16)
    if i == 5:
        pl_leg=True
    if i >= (len(tages)-2):
        xlabel=None
    if i%2 == 0:
        ylabel=None
    if tage != tages[-1]:
    #ax.axvspan(100., 911.6, color='black', alpha=0.1)
        ax.axvline(911.6, color='black', linestyle='dashed', lw=1)
        ax.axvline(504.6, color='black', linestyle='dashed', lw=1)
    nf, ms, ds = masspile(tage=tage, z=z, masslims=masslims, cname='CMRmap', alpha=0.95,
                          pl_legend=pl_leg, fontsize=14, markerfirst=True,
                          xlabel=xlabel, ylabel=ylabel,
                          ax=ax, do_title='', plt_cum=True, colors=colors,
                          xlim=(250., 10050.), ylim=(5.e-5, 7.))
    ax.annotate(r'${0:.0f}$ Myr'.format(tage*1.e9/1.e6),
                xy=(0.98, 0.98),xycoords='axes fraction', size=22, ha='right', va='top')
fig.subplots_adjust(hspace=0, wspace=0.0)
plt.setp([a.get_xticklabels() for a in fig.axes[0:-2]], visible=False)
plt.setp([a.get_yticklabels() for a in fig.axes[1::2]], visible=False)
fig.subplots_adjust(left=0.1, right=0.98, top=0.98, bottom=0.12)
fig.savefig('./f1.pdf', dpi=300)

# Figure 2

In [2]:
fsp = fsps.StellarPopulation(zcontinuous=1)
ages = fsp.ssp_ages
wavelengths = fsp.wavelengths
lsun = 3.839e33

In [ ]:
mpl.rc('legend', labelspacing=0.1, handlelength=2, fontsize=16)

In [ ]:
fig, ax = plt.subplots(1, figsize=(6.25,5))
Zs = np.array([-2.0, -1.5, -1.0, -0.5, 0.0, 0.2])
sM = getColors(np.linspace(-2.0, 0.2), maxv=0.6)
savz = []
savz.append(ages)
for z in Zs:
    fsp.params['logzsol'] = z
    wav, nuspecs = fsp.get_spectrum(peraa=False)
    wav, specs = fsp.get_spectrum(peraa=True)
    Qs = np.array([np.log10(calcQ(wav, s*lsun, f_nu=True)) for s in nuspecs])
    ax.plot(ages, Qs, color=sM.to_rgba(z), lw=2, label=r'{0: .1f}'.format(z))
    a = plt.axes([.21, .23, .33, .33])
    a.plot(10.**ages/1.e6, 10.**Qs/1.e46, color=sM.to_rgba(z), lw=2)
    a.set_xlim(0.3, 6.5)
    a.set_xlabel(r'Age [ Myr ]', size=14)
    a.set_ylim(0.1, 8.)
    a.set_ylabel(r'$\hat{Q}_{\mathrm{H}} \cdot 10^{-46}$', size=14)
    a.set_yticks([2.,4.,6.])
    savz.append(Qs)

ax.set_ylabel(r'$\log$ $\hat{Q}_{\mathrm{H}}$ [ s$^{-1}$ $\mathrm{M}_{\odot}^{-1}$]', size=16)
ax.set_xlabel(r'log Age [ years ]', size=16)
ax.set_xlim(5.9, 8.1)
ax.set_ylim(40.8, 47.2)

ax2 = ax.twiny()
ax2.set_xlabel(r'Age [ Myr ]', size=14)
desired_tick_locations = np.array([1, 3, 5, 10, 100])
def tick_function(X):
    V = np.log10(X * 1.e6)
    return V
tick_locations = tick_function(desired_tick_locations)
def tick_labels(V):
    return [r"${0:.0f}$".format(z) for z in V]
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks(tick_locations)
ax2.set_xticklabels(tick_labels(desired_tick_locations))

leg = ax.legend(loc=1, title=r'log Z/Z$_{\odot}$', fontsize=16, frameon=False)
plt.setp(leg.get_title(), fontsize=16)
fig.subplots_adjust(left=0.12, bottom=0.12, top=0.90, right=0.98)
fig.savefig('./f2.pdf', dpi=300)

In [ ]:
output = np.column_stack([savz])
head = "age "+" ".join(["{0:.2f}".format(z) for z in Zs])
print head
np.savetxt('./finalFigs/gnva_Q.txt', output, header=head)

# Figure 3

In [ ]:
def calc_slope(flu_arr, lam1, lam2):
    inds = [find_nearest(fsp.wavelengths, wav) for wav in [lam1, lam2]]
    flus = [flu_avg(flu_arr, ind) for ind in inds]
    slope = np.log10(flus[0]/flus[1])/np.log10(lam1/lam2)
    return slope

def find_nearest(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx

def flu_avg(farr, i, buff=1):
    return np.median([farr[i-buff], farr[i], farr[i+buff]])

In [ ]:
fig, ax = plt.subplots(1, figsize=(6.25,5))
Zs = np.array([-2.0, -1.5, -1.0, -0.5, 0.0, 0.2])
sM = getColors(np.linspace(-2.0, 0.2), maxv=0.6)
savz = []
savz.append(ages)
for z in Zs:
    fsp.params['logzsol'] = z
    wav, specs = fsp.get_spectrum(peraa=True)
    wavs = [505., 912.]
    slopes = [calc_slope(s, wavs[0], wavs[1]) for s in specs]
    ax.plot(ages, slopes, color=sM.to_rgba(z), lw=2, label=r'{0: .1f}'.format(z))
    savz.append(slopes)
    
ax.axvspan(6.57,6.73, color='black', alpha=0.2)
ax.annotate(r'WR', xy=(0.3,0.9), xycoords='axes fraction', size=18, color='black')

ax.set_xlim(5.9, 8.1)
ax.set_ylim(-3, 21)
leg=ax.legend(loc=4, title=r'log Z/Z$_{\odot}$', fontsize=16, frameon=False)
plt.setp(leg.get_title(), fontsize=16)

ax2 = ax.twiny()
ax2.set_xlabel(r'Age [ Myr ]', size=14)
desired_tick_locations = np.array([1, 3, 5, 10, 100])
def tick_function(X):
    V = np.log10(X * 1.e6)
    return V
tick_locations = tick_function(desired_tick_locations)
def tick_labels(V):
    return [r"${0:.0f}$".format(z) for z in V]
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks(tick_locations)
ax2.set_xticklabels(tick_labels(desired_tick_locations))

ax.set_ylabel(r'EUV slope [I$_{\mathrm{He}}$, I$_{\mathrm{H}}$]', size=16)
ax.set_xlabel(r'log Age [ years ]', size=16)
fig.subplots_adjust(left=0.12, bottom=0.12, top=0.90, right=0.98)
fig.savefig('./f3.pdf', dpi=300)

In [ ]:
output = np.column_stack([savz])
head = "age "+" ".join(["{0:.2f}".format(z) for z in Zs])
print head
np.savetxt('./finalFigs/gnva_EUV.txt', output, header=head)

# Figure 4

In [ ]:
spdict = dict(add_neb_emission=False, logzsol=0.0, gas_logz=0.0, gas_logu=-2.0)
fsp = fsps.StellarPopulation(zcontinuous=1, sfh=0, const=0.0, **spdict)
csp = fsps.StellarPopulation(zcontinuous=1, sfh=1, const=1.0)

In [ ]:
Qs = []
cQs = []
lsun = 3.839e33
for s,c in zip(specs,cspecs):
    Q = calcQ(wav, s*lsun, f_nu=True)
    Qs.append(np.log10(Q))
    cQ = calcQ(wav, c*lsun, f_nu=True)
    cQs.append(np.log10(cQ))
ages = fsp.ssp_ages
wavelengths = fsp.wavelengths

In [ ]:
fig, axes = plt.subplots(2, figsize=(6.25,8))
Zs = np.array([-2.0, -1.5, -1.0, -0.5, 0.0, 0.2])
sM = getColors(np.linspace(-2.0, 0.2), maxv=1.0, minv=0.4, cname='Blues')
sMc = getColors(np.linspace(-2.0, 0.2), maxv=1.0, minv=0.4, cname='Greens')
savz = []
savz.append(ages)
ax = fig.axes[0]

for z in Zs:
    if z == Zs[3]:
        lab = r'Burst'
    else:
        lab = '__nolegend__'
    fsp.params['logzsol'] = z
    wav, specs = fsp.get_spectrum(peraa=True)
    Qs = np.array([np.log10(calcQ(wav, s*lsun, f_nu=False)) for s in specs])
    ax.plot(ages, Qs, color=sM.to_rgba(z), lw=2, label=lab)
    if z == Zs[3]:
        lab = r'CSFR'
    else:
        lab = '__nolegend__'
    csp.params['logzsol'] = z
    mnorm = np.array([simps(csp.sfr[0:i+1], x=10.**csp.ssp_ages[0:i+1]) for i in range(len(csp.ssp_ages))])
    wav, specs = csp.get_spectrum(peraa=True)
    Qs = np.array([np.log10(calcQ(wav, s*lsun/m, f_nu=False)) for s,m in zip(specs, mnorm)])
    ax.plot(ages, Qs, color=sMc.to_rgba(z), lw=2, label=lab)
    
ax.legend(loc=0, fontsize=18, frameon=False)
ax.set_ylabel(r'$\log$ $\hat{Q}_{\mathrm{H}}$ [ s$^{-1}$ $\mathrm{M}_{\odot}^{-1}$]', size=16)
ax.set_xlabel(r'log Age [ years ]', size=16)
ax.set_xlim(5.9, 7.1)
ax.set_ylim(43.8, 47.2)
ax.set_yticks([44.0, 45.0, 46.0, 47.0])

ax2 = ax.twiny()
ax2.set_xlabel(r'Age [ Myr ]', size=14)
desired_tick_locations = np.array([1, 3, 5, 10])
def tick_function(X):
    V = np.log10(X * 1.e6)
    return V
tick_locations = tick_function(desired_tick_locations)
def tick_labels(V):
    return [r"${0:.0f}$".format(z) for z in V]

ax2.set_xlim(ax.get_xlim())
ax2.set_xticks(tick_locations)
ax2.set_xticklabels(tick_labels(desired_tick_locations))

plt.setp(fig.axes[0].get_xticklabels(), visible=False)
ax = fig.axes[1]

#fig, ax = plt.subplots(1, figsize=(6.25,5))
Zs = np.array([-2.0, -1.5, -1.0, -0.5, 0.0, 0.2])
sM = getColors(np.linspace(-2.0, 0.2), maxv=1.0, minv=0.4, cname='Blues')
sMc = getColors(np.linspace(-2.0, 0.2), maxv=1.0, minv=0.4, cname='Greens')
savz = []
savz.append(ages)
for z in Zs:
    if z == Zs[3]:
        lab = r'Burst'
    else:
        lab = '__nolegend__'
    lab = r'{0:.1f}'.format(z)
    fsp.params['logzsol'] = z
    wav, specs = fsp.get_spectrum(peraa=True)
    wavs = [505., 912.]
    slopes = [calc_slope(s, wavs[0], wavs[1]) for s in specs]
    ax.plot(ages, slopes, color=sM.to_rgba(z), lw=2, label=lab)
    #if z == Zs[3]:
    #    lab = r'CSFR'
    #else:
    lab = '__nolegend__'
    csp.params['logzsol'] = z
    wav, specs = csp.get_spectrum(peraa=True)
    mnorm = np.array([simps(csp.sfr[0:i+1], x=10.**csp.ssp_ages[0:i+1]) for i in range(len(csp.ssp_ages))])
    wavs = [505., 912.]
    slopes = [calc_slope(s/m, wavs[0], wavs[1]) for s,m in zip(specs, mnorm)]
    ax.plot(ages, slopes, color=sMc.to_rgba(z), lw=2, label=lab)

ax.set_xlim(5.9, 7.1)
ax.set_ylim(-1.2, 11)
#ax.legend(loc=0, fontsize=16, frameon=False)
leg = ax.legend(loc=0, frameon=False, fontsize=16, title=r'log Z/Z$_{\odot}$')
plt.setp(leg.get_title(), fontsize=16)

ax.set_ylabel(r'EUV slope [I$_{\mathrm{He}}$, I$_{\mathrm{H}}$]', size=16)
ax.set_xlabel(r'log Age [ years ]', size=16)
fig.subplots_adjust(left=0.12, bottom=0.08, top=0.92, right=0.98, hspace=0)
fig.savefig('./f4.pdf', dpi=300)